# Magnitude Judgement

In [ ]:
# please provide your OPENAI_KEY
OPENAI_KEY = "sk-<your-openai-key>"

# Related Functions

In [ ]:
import json
import re
import requests
import base64
import numpy as np
import pandas as pd

def run_local_vision_request(text, image_urls, temperature=0):
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    api_key = OPENAI_KEY

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": text},
            ]
        }
    ]

    for image_path in image_urls:
        # Getting the base64 string
        base64_image = encode_image(image_path)
        messages[0]["content"].append({
            "image_url": f"data:image/jpeg;base64,{base64_image}"
        })

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": temperature,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def run_test(name, description, prompt, image_urls, logger, times=1):
    response_list = []
    while len(response_list) < times:
        try:
            response = run_local_vision_request(
                text=prompt, 
                image_urls=image_urls,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            if matched_content:
                json_format_data = json.loads(matched_content.group(1))
            else: 
                json_format_data = ""
            logger.log(name, description, prompt, image_urls, response["choices"][0]["message"]["content"], json_format_data)
            response_list.append(response["choices"][0]["message"]["content"])
        except Exception as e:
            print(e)
            
    return response_list

class Logger:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, json_format_data):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "json_format_data": json_format_data}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)
logger = Logger("./magnitude.json")

## Experiment 1

### Data Preparation

In [ ]:
material_dir = './experiment_material/e1-dot-plots-axes/'
list1 = pd.read_csv(f'{material_dir}list1.csv',sep=',',header=0,index_col=['item_no'])
list2 = pd.read_csv(f'{material_dir}list2.csv',sep=',',header=0,index_col=['item_no'])

item_list = [1, 11, 2, 32, 8, 31, 4, 7, 35, 3, 14, 24, 20, 22, 15, 9, 21, 28, 23, 17]

def get_image_urls(index):
    if list1['condition'][index] == "mid":
        return []
    list1_img_url = material_dir + list1['img'][index]
    list2_img_url = material_dir + list2['img'][index]
    if list1['condition'][index] == 'hi':
        return [list1_img_url, list2_img_url]
    else: 
        return [list2_img_url, list1_img_url]   

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
def TASK(index):
    return list1['scenario'][index]+'\n'+list1['graph_cat'][index]+'\n'+'Task: '+'\n'+list1['if_phrase'][index] +'\n'+list1['chance_cat'][index]+' Answer on a scale from 1(very unlikely) to 7(very likely).' + '\n' + list1['severity_cat'][index]+' Answer on a scale from 1(very mild) to 7(very severe).' + \
        "For each question, please rate each graph with only one score." + \
        'Rate the following two graphs separately and give the reason.'
JSON_FORMAT = "Please give an additional scoring result in json format at the end of your answe, like ```json{{'chance': [score(for graph1), score(for graph2)], 'severity': [score(for graph1), score(for graph2)] }}```."

In [ ]:
name = "magnitude_experience1"
for item_no in item_list:
    description = item_no
    prompt = TASK(item_no) + JSON_FORMAT
    image_urls = get_image_urls(item_no)
    if len(image_urls) > 0:
        run_test(name, description, prompt, image_urls, logger, times=5)

## Experiment 2

### Data Preparation

In [ ]:
material_dir = './experiment_material/e2-dot-plots-axes/'
list1 = pd.read_csv(f'{material_dir}list1.csv',sep=',',header=0,index_col=['item_no'])
list2 = pd.read_csv(f'{material_dir}list2.csv',sep=',',header=0,index_col=['item_no'])

item_list = [1, 11, 2, 32, 8, 31, 4, 7, 35, 3, 14, 24, 20, 22, 15, 9, 21, 28, 23, 17]

def get_image_urls(index):
    if list1['condition'][index] == "mid":
        return []
    list1_img_url = material_dir + list1['img'][index]
    list2_img_url = material_dir + list2['img'][index]
    if list1['condition'][index] == 'hi':
        return [list1_img_url, list2_img_url]
    else: 
        return [list2_img_url, list1_img_url]  

### Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
def TASK(index):
    return list1['scenario'][index]+'\n'+list1['graph_cat'][index]+'\n'+'Task: '+'\n'+list1['if_phrase'][index] +'\n'+list1['chance_cat'][index]+' Answer on a scale from 1(very unlikely) to 7(very likely).' + '\n' + list1['severity_cat'][index]+' Answer on a scale from 1(very mild) to 7(very severe).' + \
        "For each question, please rate each graph with only one score." + \
        'Notice that he arrow on the "Chance" axis points downwards, meaning the numbers get bigger as the axis goes down.' + \
        'Rate the following two graphs separately and give the reason.'
JSON_FORMAT = "Please give an additional scoring result in json format at the end of your answe, like ```json{{'chance': [score(for graph1), score(for graph2)], 'severity': [score(for graph1), score(for graph2)] }}```."

In [ ]:
name = "magnitude_experience2"
for item_no in item_list:
    description = item_no
    prompt = TASK(item_no) + JSON_FORMAT
    image_urls = get_image_urls(item_no)
    if len(image_urls) > 0:
        run_test(name, description, prompt, image_urls, logger, times=5)